In [ ]:
import csv
import re
import selenium
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.common.exceptions import TimeoutException, WebDriverException

In [ ]:
def get_sitemap(url):
    get_url = requests.get(url)

    if get_url.status_code == 200:
        return get_url.text
    else:
        print ('Unable to fetch sitemap: %s.' % url)


def process_sitemap(s):
    soup = BeautifulSoup(s, "lxml")
    result = []

    for loc in soup.findAll("loc"):
        result.append(loc.text)

    return result


def is_sub_sitemap(s):
    if s.endswith('.xml') and 'sitemap' in s:
        return True
    else:
        return False


def parse_sitemap(s):
    sitemap = process_sitemap(s)
    result = []

    while sitemap:
        candidate = sitemap.pop()

        if is_sub_sitemap(candidate):
            sub_sitemap = get_sitemap(candidate)
            for i in process_sitemap(sub_sitemap):
                sitemap.append(i)
        else:
            result.append(candidate)

    return result

def only_articles(a):
    only_articles = []
    
    for elem in a:
        if re.search(r"article", elem):
            only_articles.append(elem)
    return only_articles


In [ ]:
EXCLUDED_URLS = ['https://www.example.com', 'https://www.example.com']

sitemap = get_sitemap('https://www.example.com/sitemap.xml')
result = parse_sitemap(sitemap)
articles = only_articles(result)

driver = webdriver.Firefox()
#agent = driver.execute_script("return navigator.userAgent")
driver.set_page_load_timeout(20)
#print("User agent: "+agent)
with open('linkchecker.csv', mode='w', encoding="utf-8") as outfile:
        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)
        header_row = ["article_url", "link_text", "link_url", "status_code", "final_url"]
        writer.writerow(header_row)
        try:
            for article in articles:
                driver.get(article)
                print('ARTICLE:', article)
                driver.wait = WebDriverWait(driver, 2)
                driver.execute_script("if (document.body !== null) {window.scrollTo(0, 900);}")
                time.sleep(1)
                #bodytext = driver.find_element_by_tag_name("body").text
                bodyhtml = driver.find_element_by_tag_name("body").get_attribute('outerHTML')
                #print(bodyhtml)
                soup = BeautifulSoup(bodyhtml, "html.parser")
                links = soup.findAll("a[not(starts-with(@href,'../'))]")
                links = driver.find_elements_by_xpath("//a[starts-with(@href, 'http') and not(contains(@href, 'www.example.com'))]")
                print("We had "+str(len(links))+" ducks to feed")

                selected_urls = []
                selected_urls_text = []
                ducks = 0
                for link in links:
                    href = link.get_attribute("href")
                    if link.get_attribute("href") not in EXCLUDED_URLS:
                        selected_urls.append(href)
                        selected_urls_text.append(link.text)
                        ducks = ducks + 1
                print("We have "+str(ducks)+" ducks to feed\n")

                for url, text in zip(selected_urls, selected_urls_text):
                    print("Retrieving : "+ url)
                    print("Anchor text: "+ text)
                    row = [article, text, url, "", ""]
                    try:
                        driver.get(url)
                        print("Final url: "+ driver.current_url)
                        r = requests.head(url)
                        print(f"HTTP code: {r.status_code}\n")
                        row = [article, text, url, r.status_code, driver.current_url]
                        # prints the int of the status code
                    except requests.ConnectionError as e:
                        print("failed to connect")
                    except TimeoutException as e:
                        print("ERORR: TimeoutException")
                        # Add "TIMEOUT" instead of HTTP status code
                        pass
                    except (WebDriverException) as e:
                        print("Hello! I hate seagulls!")
                        print(e)
                        pass
                    writer.writerow(row)

        except (WebDriverException) as e:
            print("Hello I love ducks! 🦆") # I really do love ducks
            print(e)
            pass